##**NON GRADED CHALLENGE 2**

##**BY : M RIZKI HERFIAN**

##**MATERI : Custom Functions**

##**STEP 1 : IMPORT LIBRARY**

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

**Formating**

In [2]:
np.random.seed(77)
pd.options.display.float_format='{:.4f}'.format

## **STEP 2 : PROBLEMS**

* **Build 2 50x2 dimension Tensor with random binary values and their probability of 0 and 1 is 0.5**

* **Use the first generated tensor as target, and the other vector as prediction. We will use these vectors to learn custom losses and metrics.**

* **Build the following losses used for classification with custom functions:**
 * **Cross Entropy Loss**
 * **KL-Diverge**
 * **Hinge Loss**

* **Use the loss to compare the two tensors that you have built previously. Compare the result of your custom function with the default loss function provided by Keras.**

* **Build F1 score calculation with custom metrics.**

## **STEP 3 : PROBLEM SOLVING**

**Build 2 50x2 dimension Tensor with random binary values and their probability of 0 and 1 is 0.5**

In [3]:
prob1 = tf.random.uniform(
    (50, 1), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=1, name=None
)
prob2 = tf.random.uniform(
    (50, 1), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=2, name=None
)
binary1 = tf.where(prob1>0.5,1.0,0.0)
binary2 = tf.where(prob2>0.5,1.0,0.0)

**Use the first generated tensor as target, and the other vector as prediction. We will use these vectors to learn custom losses and metrics.**

In [4]:
y_true = tf.concat([prob1,binary1],1)
y_pred = tf.concat([prob2,binary2],1)

In [5]:
y_true_proba = y_true[0:,0]
y_pred_proba = y_pred[0:,0]

y_true = y_true[0:,1]
y_pred = y_pred[0:,1]

**Build the following losses used for classification with custom functions:**

 * **Cross Entropy Loss**

In [6]:
def cross_entropy(y_true:np.ndarray, y_pred:np.ndarray):
    return (-1/len(y_true))*sum((y_true*np.log(y_pred)) + ((1-y_true)*np.log(1-y_pred)))

In [7]:
print("Cross entropy Loss Custom:",cross_entropy(y_true, y_pred_proba).numpy())

Cross entropy Loss Custom: 1.0200276


In [8]:
keras_cross_entropy = tf.keras.losses.BinaryCrossentropy()
print("Cross entropy Loss Keras:",keras_cross_entropy(y_true, y_pred_proba).numpy())

Cross entropy Loss Keras: 1.020027


 * **KL-Diverge**

In [9]:
def kl_diverge(y_true_proba:np.ndarray, y_pred_proba:np.ndarray):
    return sum(y_true_proba*np.log(y_true_proba/y_pred_proba))

In [10]:
print("KL-Diverge Loss Custom:",kl_diverge(y_true_proba, y_pred_proba).numpy())

KL-Diverge Loss Custom: 15.019988


In [11]:
keras_kl_diverge = tf.keras.losses.KLDivergence()
print("KL-Diverge Loss Keras:",keras_kl_diverge(y_true_proba, y_pred_proba).numpy())

KL-Diverge Loss Keras: 15.019989


 * **Hinge Loss**

In [12]:
def hinge(y_true:np.ndarray, y_pred:np.ndarray):
    # replace 0=-1
    new_y_true = np.array([-1 if i==0 else i for i in y_true])
    new_y_pred = np.array([-1 if i==0 else i for i in y_pred])
    
    hinge_loss = np.mean([max(0, 1-x*y) for x, y in zip(new_y_true, new_y_pred)])
    return hinge_loss

In [13]:
print("Hinge Loss Custom:",hinge(y_true, y_pred))

Hinge Loss Custom: 1.16


In [14]:
keras_hinge = tf.keras.losses.Hinge()
print("Hinge Loss Keras:",keras_hinge(y_true, y_pred).numpy())

Hinge Loss Keras: 1.16


**Build F1 score calculation with custom metrics.**

In [15]:
def f1_score(y_true:np.ndarray, y_pred:np.ndarray):
    TP = sum([1 for actual, pred in zip(y_true, y_pred) if actual==1 and pred == 1])
    TN = sum([1 for actual, pred in zip(y_true, y_pred) if actual==0 and pred == 0])
    FP = sum([1 for actual, pred in zip(y_true, y_pred) if actual==0 and pred == 1])
    FN = sum([1 for actual, pred in zip(y_true, y_pred) if actual==1 and pred == 0])
    return TP/(TP+((1/2)*(FP+FN)))

In [16]:
print("F1 Score Custom:",f1_score(y_true, y_pred))

F1 Score Custom: 0.32558139534883723


In [17]:
def keras_f1_score(y_true:np.ndarray, y_pred:np.ndarray):
    TP = tf.keras.metrics.TruePositives()
    TP.update_state(y_true, y_pred)
    TP = TP.result().numpy()
    
    TN = tf.keras.metrics.TrueNegatives()
    TN.update_state(y_true, y_pred)
    TN = TN.result().numpy()
    
    FP = tf.keras.metrics.FalsePositives()
    FP.update_state(y_true, y_pred)
    FP = FP.result().numpy()
    
    FN = tf.keras.metrics.FalseNegatives()
    FN.update_state(y_true, y_pred)
    FN = FN.result().numpy()
    
    return TP/(TP+((1/2)*(FP+FN)))

In [18]:
print("F1 Score Keras:",keras_f1_score(y_true, y_pred))

F1 Score Keras: 0.32558139534883723
